### Load modules:

In [ ]:
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams["mathtext.fontset"] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
import matplotlib.pyplot as plt
from scipy.constants import elementary_charge as e, epsilon_0 as eps_0, m_e, c, m_p
from scipy.signal import savgol_filter
import copy

### Define auxiliary objects and parameters:

In [ ]:
# define transparent colormap
Reds_t = copy.deepcopy(mpl.cm.get_cmap("Reds"))
q = np.linspace(0, Reds_t.N + 3, Reds_t.N + 3)
Reds_t._init()
Reds_t._lut[:,-1] = q / 259

In [ ]:
# define coordinates
x = np.linspace(-10.0, 15.0, 1000)
y = np.linspace(-5.0, 5.0, 400)
z = np.linspace(-5.0, 5.0, 400)

In [ ]:
# define data readers
vti_reader = vtk.vtkXMLImageDataReader()
vtu_reader = vtk.vtkXMLUnstructuredGridReader()

### Plot x-y slice:

In [ ]:
# define time step
t = 74

# load data
vti_reader.SetFileName("vtk/ne/slice_xy/ne_xy_{:04d}.vti".format(t))
vti_reader.Update()
ne = np.sum(np.reshape(vtk_to_numpy(vti_reader.GetOutput().GetCellData().GetArray(0)), [2, 400, 1000]), axis=0) / 2.0
vti_reader.SetFileName("vtk/ey/slice_xy/ey_xy_{:04d}.vti".format(t))
vti_reader.Update()
ey = np.sum(np.reshape(vtk_to_numpy(vti_reader.GetOutput().GetCellData().GetArray(0)), [2, 400, 1000]), axis=0) / 2.0
time = np.round(vtk_to_numpy(vti_reader.GetOutput().GetFieldData().GetArray(0))[0], 1)

In [ ]:
# create figure
fig, ax = plt.subplots(figsize=(8, 4))

# plot data and adjust figure parameters
im_1 = ax.pcolormesh(x, y, ne, shading="auto", cmap="bone_r", vmin=0, vmax=30, zorder=1)
im_2 = ax.pcolormesh(x, y, ey**2, shading="auto", cmap=Reds_t, vmin=0, vmax=1, zorder=3)
ax.set_xlim(-10, 15)
ax.set_ylim(-5, 5)
ax.set_xlabel(r"$ x \ (\mathrm{\mu m}) $", fontsize=20, labelpad=10)
ax.set_ylabel(r"$ y \ (\mathrm{\mu m}) $", fontsize=20, labelpad=10)
ax.tick_params(axis="both", which="major", labelsize=16)
ax.axhline(0.0, linestyle="--", color="black", alpha=0.4, zorder=2)
ax.grid(linestyle="--", alpha=0.2)
ax.text(-9, 4, r"$ t = {} \ T_0 $".format(time), fontsize=20)

# add electron density colorbar
cbar_1 = plt.colorbar(im_1, ax=ax, aspect=25, pad=0.06)
cbar_1.set_label(label=r"$ n_e \ / \ n_c $", fontsize=20, labelpad=10)
cbar_1.ax.tick_params(labelsize=16)

# add electric field colorbar
cbar_2 = plt.colorbar(im_2, ax=ax, aspect=25, pad=0.03)
cbar_2.set_label(label=r"$ \left(E_y \ / \ E_0 \right)^2 $", fontsize=20, labelpad=10)
cbar_2.ax.tick_params(labelsize=16)

In [ ]:
# save figure to file
fig.savefig("./slice_xy_{:04d}.jpg".format(t), dpi=150, bbox_inches="tight")

### Plot x-z slice:

In [ ]:
# define time step
t = 74

# load data
vti_reader.SetFileName("vtk/ne/slice_xz/ne_xz_{:04d}.vti".format(t))
vti_reader.Update()
ne = np.sum(np.reshape(vtk_to_numpy(vti_reader.GetOutput().GetCellData().GetArray(0)), [400, 2, 1000]), axis=1) / 2.0
vti_reader.SetFileName("vtk/ey/slice_xz/ey_xz_{:04d}.vti".format(t))
vti_reader.Update()
ey = np.sum(np.reshape(vtk_to_numpy(vti_reader.GetOutput().GetCellData().GetArray(0)), [400, 2, 1000]), axis=1) / 2.0
time = np.round(vtk_to_numpy(vti_reader.GetOutput().GetFieldData().GetArray(0))[0], 1)

In [ ]:
# create figure
fig, ax = plt.subplots(figsize=(8, 4))

# plot data and adjust figure parameters
im_1 = ax.pcolormesh(x, z, ne, shading="auto", cmap="bone_r", vmin=0, vmax=30, zorder=1)
im_2 = ax.pcolormesh(x, z, ey**2, shading="auto", cmap=Reds_t, vmin=0, vmax=1, zorder=3)
ax.set_xlim(-10, 15)
ax.set_ylim(-5, 5)
ax.set_xlabel(r"$ x \ (\mathrm{\mu m}) $", fontsize=20, labelpad=10)
ax.set_ylabel(r"$ z \ (\mathrm{\mu m}) $", fontsize=20, labelpad=10)
ax.tick_params(axis="both", which="major", labelsize=16)
ax.axhline(0.0, linestyle="--", color="black", alpha=0.4, zorder=2)
ax.grid(linestyle="--", alpha=0.2)
ax.text(-9, 4, r"$ t = {} \ T_0 $".format(time), fontsize=20)

# add electron density colorbar
cbar_1 = plt.colorbar(im_1, ax=ax, aspect=25, pad=0.06)
cbar_1.set_label(label=r"$ n_e \ / \ n_c $", fontsize=20, labelpad=10)
cbar_1.ax.tick_params(labelsize=16)

# add electric field colorbar
cbar_2 = plt.colorbar(im_2, ax=ax, aspect=25, pad=0.03)
cbar_2.set_label(label=r"$ \left(E_y \ / \ E_0 \right)^2 $", fontsize=20, labelpad=10)
cbar_2.ax.tick_params(labelsize=16)

In [ ]:
# save figure to file
fig.savefig("./slice_xz_{:04d}.jpg".format(t), dpi=150, bbox_inches="tight")

### Plot proton energy spectrum:

In [ ]:
# define time step
t = 99

# read data
vtu_reader.SetFileName("vtk/pp/pp_{:04d}.vtu".format(t))
vtu_reader.Update()
px = vtk_to_numpy(vtu_reader.GetOutput().GetPointData().GetArray(0))
py = vtk_to_numpy(vtu_reader.GetOutput().GetPointData().GetArray(1))
pz = vtk_to_numpy(vtu_reader.GetOutput().GetPointData().GetArray(2))
w = 163308.72

# calculate energy
ek = m_p * c**2 * (np.sqrt(1.0 + px**2 + py**2 + pz**2) - 1.0) / e / 1.0e6

# calculate histogram
ene, d_e = np.linspace(0.0, 300.0, 300, retstep=True)
h = np.histogram(ek, weights=(w * np.ones(np.size(ek)) * 1000), bins=ene)

In [ ]:
# create figure
fig, ax = plt.subplots()

# plot data
ax.plot(h[1][0:-1], h[0] / d_e, color="blue")
ax.set_yscale("log")
ax.set_xlim(0, 250)
ax.set_ylim(1e7, 1e12)
ax.set_xlabel(r"$ \mathcal{E}_p \ (\mathrm{MeV}) $", fontsize=20, labelpad=10)
ax.set_ylabel(r"$ N_p $", fontsize=20, labelpad=10)
ax.tick_params(axis="both", which="major", labelsize=16)
ax.grid(linestyle="--", alpha=0.2)

In [ ]:
# save figure to file
fig.savefig("./proton_spectrum_{:04d}.jpg".format(t), dpi=150, bbox_inches="tight")